In [55]:
import numpy as np
import tensorflow as tf
import mock, collections
from tensorflow.python.util import nest
from keras import backend as K
count = 9
batch = 2
logs_path = '/Users/kalyanb/PycharmProjects/MetaLearning/while_loop_log/'
np.random.seed(7)
def _wrap_variable_creation(func, custom_getter):
  """Provides a custom getter for all variable creations."""
  original_get_variable = tf.get_variable
  def custom_get_variable(*args, **kwargs):
    if hasattr(kwargs, "custom_getter"):
      raise AttributeError("Custom getters are not supported for optimizee "
                           "variables.")
    return original_get_variable(*args, custom_getter=custom_getter, **kwargs)

  # Mock the get_variable method.
  with mock.patch("tensorflow.get_variable", custom_get_variable):
    return func()


def _get_variables(func):

  variables = []
  constants = []

  def custom_getter(getter, name, **kwargs):
    trainable = kwargs["trainable"]
    kwargs["trainable"] = False
    variable = getter(name, **kwargs)
    if trainable:
      variables.append(variable)
    else:
      constants.append(variable)
    return variable

  _wrap_variable_creation(func, custom_getter)

  return variables, constants


def _make_with_custom_variables(func, variables):

  variables = collections.deque(variables)

  def custom_getter(getter, name, **kwargs):
    if kwargs["trainable"]:
      return variables.popleft()
    else:
      kwargs["reuse"] = True
      return getter(name, **kwargs)

  return _wrap_variable_creation(func, custom_getter)


def prob():
    da = tf.random_normal(shape=[10,],dtype=tf.float32,stddev=0.01)
    with tf.name_scope("Loss"):
        def loss():
            with tf.name_scope("index"):
                index = tf.random_uniform([1], 0, 10, tf.int64)
            with tf.variable_scope("var",reuse=tf.AUTO_REUSE):
                x1 = tf.get_variable("x1",shape=[1],dtype=tf.float32,initializer=tf.zeros_initializer)
            d = tf.gather(da,index)
            return tf.add(x1,d)
    with tf.name_scope("Aux_Loss"):
        def aux_loss():
            with tf.name_scope("index"):
                index = tf.random_uniform([1], 0, 10, tf.int64)
            with tf.variable_scope("aux_var",reuse=tf.AUTO_REUSE):
                v = tf.get_variable("v",shape=[1],dtype=tf.float32,initializer=tf.zeros_initializer)
                x2 = tf.get_variable("x2",shape=[1],dtype=tf.float32,initializer=tf.zeros_initializer)
            d = tf.gather(da,index)
            return tf.add(x2,tf.add(v,d))
    return collections.OrderedDict([('loss',loss),('aux_loss',aux_loss)])


def test(prob):
    optvar = [_get_variables(a)[0] for a in prob.values()]
    num = nest.flatten([0,[len(a) for a in optvar]])

    def time_step(t,f_array,f_array_loss,f_array_aux,x):
        losstot = [_make_with_custom_variables(a, x[num[b]:num[b]+num[b+1]]) 
                   for a,b in zip(prob.values(),range(len(num)-1))]
        fx_loss = losstot[0]
        f_array_loss = f_array_loss.write(t,fx_loss)
        fx_aux =losstot[1]
        f_array_aux = f_array_aux.write(t, fx_aux)
        fx = sum(losstot[a] for a in range(len(losstot)))
        f_array = f_array.write(t, fx)
        x_n = [a+1 for a in x]
        t_n = t+1
        return t_n,f_array,f_array_loss,f_array_aux,x_n

    fx_array = tf.TensorArray(tf.float32, size=count,
                              clear_after_read=False)
    fx_array_loss = tf.TensorArray(tf.float32, size=count,
                              clear_after_read=False)
    fx_array_aux = tf.TensorArray(tf.float32, size=count,
                                   clear_after_read=False)
    t_f, fx_array, fx_array_loss, fx_array_aux, x_final = tf.while_loop(
        cond=lambda t, *_: t < count-1,
        body=time_step,
        loop_vars=(0, fx_array, fx_array_loss, fx_array_aux, nest.flatten(optvar)),
        parallel_iterations=1,
        swap_memory=True,
        name="loop")
    finallosstot = [_make_with_custom_variables(a,x_final[num[b]:num[b]+num[b+1]]) 
                    for a,b in zip(prob.values(),range(len(num)-1))]
    fx_final = sum(finallosstot[a] for a in range(len(finallosstot)))
    fx_final_loss = finallosstot[0]
    fx_final_aux = finallosstot[1]
    with tf.name_scope('WriteFinalAux'):
        fx_array_aux = fx_array_aux.write(count-1,fx_final_aux)
    with tf.name_scope('WriteFinalLoss'):
        fx_array_loss = fx_array_loss.write(count-1,fx_final_loss)
    with tf.name_scope("WriteFinal"):
        fx_array = fx_array.write(count-1,fx_final)
    with tf.name_scope("ArrayInfo"):
        array_f= fx_array.stack()
    with tf.name_scope("ArrayInfoLoss"):
        array_f_loss= fx_array_loss.stack()
    with tf.name_scope("ArrayInfoAux"):
        array_f_aux= fx_array_aux.stack()
    with tf.name_scope("Reset"):
        reset = [tf.variables_initializer(nest.flatten(optvar)),
                 fx_array.close(),fx_array_aux.close(),fx_array_loss.close()]
    with tf.name_scope("Update"):
        update = [tf.assign(r,v) for r,v in zip(nest.flatten(optvar),x_final)]

    return update,reset, array_f, t_f, fx_final, x_final, array_f_loss, array_f_aux

prob = prob()
update, reset, array_f, t_f, fx_final, x_final, array_f_loss, array_f_aux = test(prob)
with tf.Session() as sess:
    graph_writer = tf.summary.FileWriter(logs_path, sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(reset)
    fall = []
    for i in range(2):
        t,xf,f,f_loss,f_aux,_ = sess.run([t_f,x_final,array_f, array_f_loss, array_f_aux, update])
        print(f - (f_loss + f_aux))
        print(f_aux)
        fall.append(f)
    graph_writer.close()

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[ -9.72272828e-03]
 [  1.99660659e+00]
 [  3.99027729e+00]
 [  5.99027729e+00]
 [  7.99189663e+00]
 [  9.99660683e+00]
 [  1.20066261e+01]
 [  1.40066261e+01]
 [  1.59993162e+01]]
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[ 16.00154114]
 [ 18.02223969]
 [ 19.99732018]
 [ 21.9932003 ]
 [ 24.02223969]
 [ 25.9932003 ]
 [ 27.99959946]
 [ 29.99959946]
 [ 32.00499725]]
